# Processing the Airline Reporting Carrier On-Time Performance Dataset

This notebook relates to the Airline Reporting Carrier On-Time Performance Dataset. The dataset  contains information on approximately 200 million domestic US flights reported to the United States Bureau of Transportation Statistics, from 1987 - 2020. This dataset is freely available from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/airline/).

**Note** the full dataset is very large (over 80GB uncompressed), so here we work with a smaller sample dataset containing a total of 2 million rows.

In this notebook, we process the raw dataset by:
* selecting the columns we wish to keep for later analysis
* converting and cleaning data where required
* handling missing values

#### Import required modules

Import and configure the required modules.

In [1]:
# !pip install pandas > /dev/null 2>&1

In [2]:
# Define required imports
import pandas as pd
# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Read the Raw Data

We start by reading in the raw dataset and displaying the first few rows of the dataframe.

In [3]:
data_path = 'data/airline_2m.csv'
raw_data = pd.read_csv(data_path, encoding = "ISO-8859-1", parse_dates=['FlightDate'],
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
raw_data.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,...,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1998,1,1,2,5,1998-01-02,NW,19386,NW,N297US,675,13487,1348701,31650,MSP,"Minneapolis, MN",MN,27.0,Minnesota,63,14869,1486902,34614,SLC,"Salt Lake City, UT",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,2,5,28,4,2009-05-28,FL,20437,FL,N946AT,671,13342,1334202,33342,MKE,"Milwaukee, WI",WI,55.0,Wisconsin,45,13204,1320401,31454,MCO,"Orlando, FL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,2,6,29,6,2013-06-29,MQ,20398,MQ,N665MQ,3297,11921,1192102,31921,GJT,"Grand Junction, CO",CO,8.0,Colorado,82,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,3,8,31,2,2010-08-31,DL,19790,DL,N6705Y,1806,12892,1289201,32575,LAX,"Los Angeles, CA",CA,6.0,California,91,11433,1143301,31295,DTW,"Detroit, MI",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,1,1,15,7,2006-01-15,US,20355,US,N504AU,465,11618,1161801,31703,EWR,"Newark, NJ",NJ,34.0,New Jersey,21,11057,1105702,31057,CLT,"Charlotte, NC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean the Data

Fortunately, the airline delay dataset is relatively clean already! The fields we wish to use already represent variables such as unique codes for the airline, origin and destination. There are also fields representing binned variables for departure time slot and flight distance.

We will select a subset of the data relating to years 2010-2017, with origin airport `JFK`, to match our weather data. For simplicity, we will focus on delayed flights and ignore flight cancellations. We will ignore arrival delays, hence we will be focusing on predicting "departure delays" _from JFK_ to other destinations, using the field `DepDel15`. This is a binary value indicating whether the flight was delayed by more than 15 minutes (deemed to be `delayed`) or not (deemed to be `on time`).

In [4]:
# Select the data sub-set for years 2010-2017 for flights originating from JFK
jfk_flights = raw_data.copy()
jfk_flights = jfk_flights[(jfk_flights['Origin'] == 'JFK') & (jfk_flights['Year'].isin(range(2010, 2018))) & (jfk_flights['Cancelled'] == 0)]
jfk_flights.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,...,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
66,2015,1,3,17,2,2015-03-17,B6,20409,B6,N247JB,1729,12478,1247802,31703,JFK,"New York, NY",NY,36.0,New York,22,14635,1463502,31714,RSW,"Fort Myers, FL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2016,2,5,25,3,2016-05-25,B6,20409,B6,N907JB,1103,12478,1247803,31703,JFK,"New York, NY",NY,36.0,New York,22,14843,1484304,34819,SJU,"San Juan, PR",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,2017,3,7,26,3,2017-07-26,AA,19805,AA,N792AA,9,12478,1247803,31703,JFK,"New York, NY",NY,36.0,New York,22,14771,1477102,32457,SFO,"San Francisco, CA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509,2010,4,12,11,6,2010-12-11,B6,20409,B6,N593JB,195,12478,1247801,31703,JFK,"New York, NY",NY,36.0,New York,22,12889,1288902,32211,LAS,"Las Vegas, NV",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,2012,3,8,11,6,2012-08-11,MQ,20398,MQ,N605MQ,4309,12478,1247802,31703,JFK,"New York, NY",NY,36.0,New York,22,12339,1233904,32337,IND,"Indianapolis, IN",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We create a mapping of airline id to a more readable airline name (see the [airline ID looukp table](https://www.transtats.bts.gov/Download_Lookup.asp?Lookup=L_AIRLINE_ID)).

In [5]:
airline_codes = {
    20409: 'JetBlue',
    19790: 'Delta',
    19805: 'American Airlines',
    20398: 'Envoy Air',
    19977: 'United',
    21171: 'Virgin America',
    20363: 'Endeavor Air',
    20417: 'Comair',
    20355: 'US Airways',
    20366: 'ExpressJet',
    19690: 'Hawaiian Airlines',
    19930: 'Alaska Airlines',
    20378: 'Mesa Airlines'
}

In [6]:
jfk_flights.loc[:, 'airline_name'] = jfk_flights['DOT_ID_Reporting_Airline'].map(lambda x: airline_codes[x])

Next, we select the set of columns to keep for downstream analysis and rename the columns to `snake_case` for consistency with our processed weather dataset.

In [7]:
cols_to_keep = ['FlightDate', 'Month', 'DayofMonth', 'DayOfWeek', 'DOT_ID_Reporting_Airline', 'airline_name', 'Origin', 'Dest', 'CRSDepTime', 'DepTimeBlk', 'DistanceGroup', 'DepDel15', 'DepDelay']
jfk_flights = jfk_flights[cols_to_keep]
col_names = {
    'FlightDate': 'flight_date',
    'Month': 'month',
    'DayofMonth': 'day_of_month',
    'DayOfWeek': 'day_of_week',
    'DOT_ID_Reporting_Airline': 'airline_id',
    'Origin': 'origin',
    'Dest': 'dest',
    'CRSDepTime': 'sched_dep_time',
    'DepTimeBlk': 'dep_time_bin',
    'DistanceGroup': 'distance_bin',
    'DepDel15': 'delayed',
    'DepDelay': 'dep_delay'
}
jfk_flights_renamed = jfk_flights.rename(columns=col_names)
jfk_flights_renamed.head()

,flight_date,month,day_of_month,day_of_week,airline_id,airline_name,origin,dest,sched_dep_time,dep_time_bin,distance_bin,delayed,dep_delay
66,2015-03-17,3,17,2,20409,JetBlue,JFK,RSW,1525,1500-1559,5,0.0,-7.0
70,2016-05-25,5,25,3,20409,JetBlue,JFK,SJU,2119,2100-2159,7,0.0,0.0
283,2017-07-26,7,26,3,19805,American Airlines,JFK,SFO,700,0700-0759,11,0.0,-3.0
509,2010-12-11,12,11,6,20409,JetBlue,JFK,LAS,1900,1900-1959,9,0.0,-5.0
518,2012-08-11,8,11,6,20398,Envoy Air,JFK,IND,1510,1500-1559,3,0.0,-2.0


In [8]:
# Log some general information about the dataset
print('# of columns: ' + str(jfk_flights_renamed.shape[1])) 
print('# of observations: ' + str(jfk_flights_renamed.shape[0]))
print('Start date: ' + str(jfk_flights_renamed['flight_date'].min()))
print('End date: ' + str(jfk_flights_renamed['flight_date'].max()))

# of columns: 13
# of observations: 8105
Start date: 2010-01-01 00:00:00
End date: 2017-12-31 00:00:00


### Save the Processed Data

Finally, we save the processed dataset for use by downstream tasks.

In [9]:
jfk_flights_renamed.to_csv('data/jfk_flight_features.csv', index=False, float_format='%g')

 ### Authors
 
 This notebook was created by the [Center for Open-Source Data & AI Technologies](http://codait.org).

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.